In [52]:
from PIL import Image
import numpy as np
import os
os.chdir("C:\\Users\\TakaiYohei\\Desktop\\pix2pix")
import chainer
from chainer import Variable
from net import Encoder
from net import Decoder

In [53]:
# Set up a neural network
enc = Encoder(in_ch=12)
dec = Decoder(out_ch=3)

In [54]:
# load terained models
chainer.serializers.load_npz("./result/enc_iter_50000.npz", enc)
chainer.serializers.load_npz("./result/dec_iter_50000.npz", dec)

In [65]:
# load input
label = Image.open("./facade/base/cmp_b0001.png")
w,h = label.size
r = 286 / float(min(w,h))
# resize images so that min(w, h) == 286
w = int(r*w)
h = int(r*h)
label = label.resize((w, h), Image.NEAREST)

In [66]:
label_ = np.asarray(label)-1  # [0, 12)
print(label_.shape)
label = np.zeros((1, 12, h, w)).astype("i") # (12(ラベルの数), H, W)のint型
for j in range(12):
    label[0,j,:] = label_==j

(539, 286)


In [67]:
crop_width = 256
x_l = np.random.randint(0,w-crop_width)
x_r = x_l+crop_width
y_l = np.random.randint(0,h-crop_width)
y_r = y_l+crop_width
label = label[:,:,y_l:y_r,x_l:x_r]
label = Variable(label.astype("f"))

In [68]:
# encode & decode
x = dec(enc(label))

In [69]:
# save output image
print(x.shape)
_, C, H, W = x.shape
x = x.reshape((C, H, W))
print(x.shape)
x = x.transpose(1, 2, 0)

(1, 3, 256, 256)
(3, 256, 256)


In [70]:
x = x * 128+128

In [71]:
x = x.data

In [72]:
x = np.clip(x, 0.0, 255.0)

In [73]:
x = x.astype('uint8')

In [74]:
Image.fromarray(x).convert('RGB').save('./output.png')